In [8]:
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
sys.path.append(str("/home/alessio/dr_sasa_python/build/lib"))
sys.path.append(str("/home/alessio/dr_sasa_python/bindings/python/utils"))
import dr_sasa_py
from structure_parser import *

In [10]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/3i40.pdb"
calc = dr_sasa_py.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), chains=[["A"], ["B"]], include_matrix=True, print_output=True)

Object A complexed surface (A^2):	1198.94
Object B complexed surface (A^2):	2165.03
Object A uncomplexed surface (A^2):	1990.44
Object B uncomplexed surface (A^2):	2906.85
A <--- B buried surface (A^2):	791.494
A ---> B buried surface (A^2):	741.817
Interface A/B (A^2):	766.656


Object A complexed surface (A^2):	1201.52
Object B complexed surface (A^2):	2161.36
Object A uncomplexed surface (A^2):	1998.48
Object B uncomplexed surface (A^2):	2905.35
A <--- B buried surface (A^2):	796.957
A ---> B buried surface (A^2):	743.99
Interface A/B (A^2):	770.474

In [13]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/3i40.pdb"
calc = dr_sasa_py.DecoupledSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), print_output=True)

In [ ]:
pd.read_csv("/home/alessio/dr_sasa_python/tests/test_results/3i40.atmasa", sep="\t")
pd.read_csv("/home/alessio/dr_sasa_python/my_analysis", sep="\t")

,atmname,resname,chain,resnum,ID,total_ASA,bb_ASA,sc_ASA,majorgroove_ASA,minorgroove_ASA,...,hyd_sc_ASA,hyd_majorgroove_ASA,hyd_minorgroove_ASA,hyd_nogroove_ASA,lig_ASA,lig_polar_ASA,lig_hyd_ASA,X,Y,Z
0,N,GLY,A,1,1,27.304,27.304,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-27.279,6.238,-12.314
1,CA,GLY,A,1,2,13.426,13.426,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-26.249,6.028,-11.313
2,C,GLY,A,1,3,0.008,0.008,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-25.582,4.677,-11.471
3,O,GLY,A,1,4,0.000,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-25.731,4.023,-12.501
4,N,ILE,A,2,5,0.434,0.434,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-24.853,4.248,-10.446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,CA,ALA,B,30,408,12.242,12.242,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-28.366,8.568,-6.616
399,C,ALA,B,30,409,4.066,4.066,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-27.782,8.115,-7.955
400,O,ALA,B,30,410,29.604,29.604,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-27.485,8.928,-8.831
401,CB,ALA,B,30,411,47.642,NaN,47.642,NaN,NaN,...,47.642,NaN,NaN,NaN,NaN,NaN,NaN,-27.302,9.269,-5.772


In [ ]:
total_sasa = result["sasa"]["values"].sum()
per_residue_sasa = {}
for i, resname in enumerate(result["atom_info"]["residue_names"]):
    per_residue_sasa[resname] = result["sasa"]["values"][i]

# Molecular composition
mol_composition = result["molecular"]["atoms_by_type"]
print(f"Molecule contains: {mol_composition}")

# Interface analysis (for applicable modes)
if "interface" in result:
    print(f"Total interface area: {result['interface']['total_area']}")
    print(f"Buried surface area: {result['interface']['buried_surface_area']}")